# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **What percentage of respondents work remotely vs. in-office?**
5. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
6. **What's the salary gap between men and women in tech roles?**
7. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.

---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

### ✅ Data Loading & Exploration
- [✓] Load TSV file with proper encoding and delimiters
- [✓] Initial data exploration and shape analysis
- [✓] Column inspection and naming standardization

### ✅ Data Cleaning Pipeline
- [✓] Standardize column names to snake_case
- [✓] Parse and clean salary data (remove commas, convert to numeric)
- [✓] Handle missing values with appropriate imputation strategies
- [✓] Normalize currency codes and country names
- [✓] Create US scope filtering logic
- [✓] Standardize industry categories (fix fragmentation issues)
- [✓] Create tech industry classification

### ✅ Business Questions Analysis
- [✓] Q1: Software Engineer median salary (regex pattern matching)
- [✓] Q2: Highest paying US state for tech workers
- [✓] Q3: Salary increase per year of experience (linear regression)
- [✓] Q4: Remote vs in-office work percentage
- [✓] Q5: Highest paying non-tech industry (with standardized categories)
- [✓] Bonus: Gender salary gap analysis
- [✓] Bonus: Education level premium analysis

### Key Insights Discovered
- Industry standardization critical (reduced 1,220 → 488 categories)
- Pharmaceuticals outperform general tech ($115K vs $90K median)
- Software Engineers earn $140K median (premium subset of tech)
- $2,324 salary increase per year of experience in tech

## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.

In [22]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression
from pathlib import Path
from typing import Dict, Optional

# Suppress warnings for cleaner notebook output
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings('ignore', category=FutureWarning, module='pandas')

print("✅ Libraries imported and warnings suppressed")

✅ Libraries imported and warnings suppressed


In [23]:
# Load the Ask A Manager Salary Survey (TSV) robustly
filename = 'Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey2021 (Responses) - Form Responses 1.tsv'
repo_root = Path('/Users/akkeem/Documents/ClassAssignments/TTPR/Week-05/ds-fall-2025-fri-0630')

# Search for the file under repo_root
candidates = list(repo_root.rglob(filename))
if not candidates:
    candidates = list(Path('.').rglob(filename))
if not candidates:
    raise FileNotFoundError(f"Could not find '{filename}' under {repo_root} or current working dir")

data_path = candidates[0]
print('Loading from:', data_path)

# Load TSV file
df = pd.read_csv(data_path, sep='\t', encoding='utf-8', low_memory=False)

# Initial exploration
print('Dataset shape:', df.shape)
print('\nColumn preview:')
print(df.columns.tolist()[:10])
df.head()

Loading from: /Users/akkeem/Documents/ClassAssignments/TTPR/Week-05/ds-fall-2025-fri-0630/Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey2021 (Responses) - Form Responses 1.tsv
Dataset shape: (28062, 18)

Column preview:
['Timestamp', 'How old are you?', 'What industry do you work in?', 'Job title', 'If your job title needs additional context, please clarify here:', "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)", 'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.', 'Please indicate the currency', 'If "Other," please indicate the currency here: ', 'If your income needs additional context, please provide it here:']


,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


## Step 2: Data Cleaning

Clean and standardize the messy data for analysis.

In [24]:
# STEP 2A: Column Standardization
print("🔧 COLUMN STANDARDIZATION")
print("=" * 40)

# Pre-rename long annual salary column
explicit_map = {}
for col in df.columns:
    if re.search(r"^what\s+is\s+your\s+annual\s+salary\?", col, flags=re.IGNORECASE):
        explicit_map[col] = 'annual_salary'

if explicit_map:
    df = df.rename(columns=explicit_map).copy()
    print('Pre-renamed annual salary column ✓')

# Convert to snake_case
def to_snake(name: str) -> str:
    name = name.strip().lower()
    name = re.sub(r"[\s\-/]+", "_", name)
    name = re.sub(r"[^a-z0-9_]+", "", name)
    name = re.sub(r"_+", "_", name)
    return name.strip("_")

# Build new names with de-duplication
new_columns = []
seen = {}
for col in df.columns:
    base = to_snake(col)
    if base not in seen:
        seen[base] = 0
        new_columns.append(base)
    else:
        seen[base] += 1
        new_columns.append(f"{base}_{seen[base]}")

df.columns = new_columns
print(f'Converted {len(df.columns)} columns to snake_case ✓')

# Apply intuitive, human-friendly column names (comprehensive mapping)
cols = list(df.columns)

def find_col(*keywords: str) -> Optional[str]:
    """Find the first column containing all keywords (case-insensitive)."""
    keys = [k.lower() for k in keywords]
    for c in cols:
        lc = c.lower()
        if all(k in lc for k in keys):
            return c
    return None

intuitive_map: Dict[str, str] = {}

# Timestamp / demographics / location
if (c := find_col('timestamp')): intuitive_map[c] = 'timestamp'
if (c := find_col('how', 'old')): intuitive_map[c] = 'age_group'
if (c := find_col('what', 'industry')): intuitive_map[c] = 'industry'
if (c := find_col('job', 'title')): intuitive_map[c] = 'job_title'
if (c := find_col('job', 'title', 'context')): intuitive_map[c] = 'job_title_context'
if (c := find_col('what', 'country')): intuitive_map[c] = 'country'
if (c := find_col('state')): intuitive_map[c] = 'us_state'
if (c := find_col('city')): intuitive_map[c] = 'city'

# Compensation
if (c := find_col('annual', 'salary')): intuitive_map[c] = 'annual_salary'
if (c := find_col('additional', 'compensation')): intuitive_map[c] = 'additional_compensation'
if (c := find_col('please', 'indicate', 'currency')): intuitive_map[c] = 'currency'
if (c := find_col('currency', 'other')): intuitive_map[c] = 'currency_other'
if (c := find_col('income', 'context')): intuitive_map[c] = 'income_context'

# Experience and education
if (c := find_col('years', 'overall')): intuitive_map[c] = 'years_experience_overall'
if (c := find_col('years', 'field')): intuitive_map[c] = 'years_experience_field'
if (c := find_col('highest', 'education')): intuitive_map[c] = 'highest_education'

# Demographics
if (c := find_col('gender')): intuitive_map[c] = 'gender'
if (c := find_col('race')): intuitive_map[c] = 'race'

# Apply rename
human_df = df.rename(columns=intuitive_map).copy()

# Show mapping preview
print('Intuitive rename mapping:')
for old, new in intuitive_map.items():
    print(f'  • {old} → {new}')

# Validate no collisions
assert len(set(human_df.columns)) == len(human_df.columns), 'Duplicate names after intuitive renaming'

df = human_df
print(f'\n✅ Applied {len(intuitive_map)} intuitive column names')
print(f'Column count: {len(df.columns)}')
print(f'Final columns: {list(df.columns)[:10]}...')

🔧 COLUMN STANDARDIZATION
Pre-renamed annual salary column ✓
Converted 18 columns to snake_case ✓
Intuitive rename mapping:
  • timestamp → timestamp
  • how_old_are_you → age_group
  • what_industry_do_you_work_in → industry
  • job_title → job_title
  • if_your_job_title_needs_additional_context_please_clarify_here → job_title_context
  • what_country_do_you_work_in → country
  • if_youre_in_the_us_what_state_do_you_work_in → us_state
  • what_city_do_you_work_in → city
  • annual_salary → annual_salary
  • how_much_additional_monetary_compensation_do_you_get_if_any_for_example_bonuses_or_overtime_in_an_average_year_please_only_include_monetary_compensation_here_not_the_value_of_benefits → additional_compensation
  • please_indicate_the_currency → currency
  • if_other_please_indicate_the_currency_here → currency_other
  • if_your_income_needs_additional_context_please_provide_it_here → income_context
  • how_many_years_of_professional_work_experience_do_you_have_overall → years_exper

In [25]:
# STEP 2B: Data Cleaning & Type Conversion
print("\n🧹 DATA CLEANING & TYPE CONVERSION")
print("=" * 40)

original_rows = len(df)

# Parse salary columns to numeric
for col in ['annual_salary', 'additional_compensation']:
    if col in df.columns:
        df[col] = (
            df[col].astype(str)
            .str.replace(',', '', regex=False)
            .str.extract(r'([-+]?[0-9]*\.?[0-9]+)', expand=False)
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')

print('Parsed salary columns to numeric ✓')

# Handle missing values strategically
null_summary = df.isna().mean().sort_values(ascending=False)
print(f'\nNull percentages (top 5):')
print((null_summary * 100).round(1).head())

# Fill categorical columns with 'Unknown'
categorical_cols = ['industry', 'job_title', 'country', 'us_state', 'highest_education', 'gender']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# Drop rows with excessive missingness (>40% null)
row_null_frac = df.isna().mean(axis=1)
df = df[row_null_frac <= 0.40].copy()

print(f'Dropped rows with >40% nulls: {original_rows} → {len(df)} rows ✓')
print(f'Data cleaning complete: {len(df):,} clean rows ready for analysis')


🧹 DATA CLEANING & TYPE CONVERSION
Parsed salary columns to numeric ✓

Null percentages (top 5):
currency_other             99.3
income_context             89.2
job_title_context          74.1
additional_compensation    26.0
us_state                   17.9
dtype: float64
Dropped rows with >40% nulls: 28062 → 28062 rows ✓
Data cleaning complete: 28,062 clean rows ready for analysis


In [26]:
# STEP 2C: Geographic & Currency Standardization
print("\n🌍 GEOGRAPHIC & CURRENCY STANDARDIZATION")
print("=" * 50)

# Standardize currency codes
if 'currency' in df.columns:
    df['currency'] = df['currency'].astype(str).str.strip().str.upper()
    currency_map = {
        'US DOLLARS': 'USD', 'USD$': 'USD', 'US$': 'USD', '$': 'USD',
        'U.S. DOLLARS': 'USD', 'DOLLARS': 'USD'
    }
    df['currency'] = df['currency'].replace(currency_map)
    print('Standardized currency codes ✓')

# Standardize country names
if 'country' in df.columns:
    df['country'] = df['country'].astype(str).str.strip()
    country_map = {
        'US': 'United States', 'USA': 'United States', 'U.S.': 'United States',
        'United States of America': 'United States'
    }
    df['country'] = df['country'].replace(country_map)
    print('Standardized country names ✓')

# Create US scope flag (critical for consistent analysis)
has_us_country = (df['country'] == 'United States') if 'country' in df.columns else False
has_us_state = df['us_state'].notna() if 'us_state' in df.columns else False
uses_usd = (df['currency'] == 'USD') if 'currency' in df.columns else False

# Scope logic: in US OR (USD currency AND US state provided)
df['is_us_scope'] = has_us_country | (uses_usd & has_us_state)

print(f'Created US scope flag:')
print(f'  • US country responses: {has_us_country.sum() if hasattr(has_us_country, "sum") else 0:,}')
print(f'  • USD + US state responses: {(uses_usd & has_us_state).sum() if hasattr(uses_usd, "sum") and hasattr(has_us_state, "sum") else 0:,}')
print(f'  • Total US scope: {df["is_us_scope"].sum():,} responses ✓')


🌍 GEOGRAPHIC & CURRENCY STANDARDIZATION
Standardized currency codes ✓
Standardized country names ✓
Created US scope flag:
  • US country responses: 21,761
  • USD + US state responses: 23,374
  • Total US scope: 23,395 responses ✓


In [27]:
# STEP 2D: Industry Standardization (Critical for Accurate Analysis)
print("\n🏭 INDUSTRY STANDARDIZATION")
print("=" * 40)

def standardize_industry(industry_str):
    """Standardize industry names by mapping variants to canonical forms."""
    if pd.isna(industry_str):
        return 'Unknown'
    
    industry = str(industry_str).lower().strip()
    industry = re.sub(r'\s+', ' ', industry)
    industry = re.sub(r'[^\w\s&/-]', '', industry)
    
    # Standardization rules (most specific first)
    rules = {
        'Pharmaceuticals': ['pharma', 'pharmaceutical', 'biotech', 'biopharm'],
        'Healthcare': ['health care', 'healthcare', 'medical', 'hospital', 'clinical'],
        'Finance & Banking': ['finance', 'financial', 'banking', 'investment', 'fintech'],
        'Technology': ['computing or tech', 'technology', 'tech', 'software', 'computer'],
        'Education': ['education', 'academic', 'university', 'school'],
        'Government': ['government', 'public administration', 'federal'],
        'Manufacturing': ['engineering or manufacturing', 'manufacturing', 'industrial'],
        'Consulting': ['consulting', 'management consulting'],
        'Legal': ['law', 'legal', 'attorney'],
        'Media & Communications': ['media', 'marketing', 'advertising', 'communications'],
        'Energy & Utilities': ['energy', 'utilities', 'oil', 'gas']
    }
    
    for standard_name, variants in rules.items():
        for variant in variants:
            if variant in industry:
                return standard_name
    
    return industry.title()

# Apply standardization
original_industries = df['industry'].nunique()
df['industry_standardized'] = df['industry'].apply(standardize_industry)
standardized_industries = df['industry_standardized'].nunique()

print(f'Industry standardization:')
print(f'  • Original categories: {original_industries:,}')
print(f'  • Standardized categories: {standardized_industries:,}')
print(f'  • Reduction: {((original_industries - standardized_industries) / original_industries * 100):.1f}% ✓')

# Show top standardized industries
print(f'\nTop 10 standardized industries:')
print(df['industry_standardized'].value_counts().head(10))


🏭 INDUSTRY STANDARDIZATION
Industry standardization:
  • Original categories: 1,220
  • Standardized categories: 695
  • Reduction: 43.0% ✓

Top 10 standardized industries:
industry_standardized
Technology                4754
Education                 3375
Nonprofits                2419
Media & Communications    2282
Healthcare                2219
Government                1931
Finance & Banking         1825
Manufacturing             1772
Legal                     1145
Consulting                 893
Name: count, dtype: int64
Industry standardization:
  • Original categories: 1,220
  • Standardized categories: 695
  • Reduction: 43.0% ✓

Top 10 standardized industries:
industry_standardized
Technology                4754
Education                 3375
Nonprofits                2419
Media & Communications    2282
Healthcare                2219
Government                1931
Finance & Banking         1825
Manufacturing             1772
Legal                     1145
Consulting           

In [28]:
# STEP 2E: Tech Industry Classification
print("\n💻 TECH INDUSTRY CLASSIFICATION")
print("=" * 40)

# Create tech flag using standardized industries
tech_industries = {'Technology'}
df['is_tech_industry'] = df['industry_standardized'].isin(tech_industries)

# Include biotech companies that are tech-focused
biotech_tech_keywords = ['bioinformatics', 'computational biology']
for keyword in biotech_tech_keywords:
    biotech_mask = df['industry'].str.contains(keyword, case=False, na=False)
    df.loc[biotech_mask, 'is_tech_industry'] = True

# Create normalized industry column
df['industry_normalized'] = df['industry_standardized'].copy()
df.loc[df['is_tech_industry'], 'industry_normalized'] = 'Technology'

print(f'Tech industry classification:')
print(f'  • Total responses: {len(df):,}')
print(f'  • Tech industry: {df["is_tech_industry"].sum():,}')
print(f'  • Non-tech: {(~df["is_tech_industry"]).sum():,}')
print(f'  • Tech percentage: {df["is_tech_industry"].mean()*100:.1f}% ✓')

# Create analysis-ready subsets
us_scope_df = df[df['is_us_scope']].copy()
tech_us_df = df[df['is_tech_industry'] & df['is_us_scope']].copy()

print(f'\nAnalysis-ready datasets:')
print(f'  • US scope data: {len(us_scope_df):,} responses')
print(f'  • Tech workers (US): {len(tech_us_df):,} responses')
print(f'\n✅ Data cleaning pipeline complete!')


💻 TECH INDUSTRY CLASSIFICATION
Tech industry classification:
  • Total responses: 28,062
  • Tech industry: 4,755
  • Non-tech: 23,307
  • Tech percentage: 16.9% ✓

Analysis-ready datasets:
  • US scope data: 23,395 responses
  • Tech workers (US): 3,825 responses

✅ Data cleaning pipeline complete!


## Step 3: Business Questions Analysis

Now answer those important business questions!

In [29]:
# Question 1: What is the median salary for Software Engineers in the United States?

print("QUESTION 1: Median salary for Software Engineers in the United States")
print("=" * 70)

# Software Engineer pattern (comprehensive regex)
se_pattern = r'(?i)\b(?:software|sr\.?\s*software|senior\s+software|junior\s+software|lead\s+software|principal\s+software)\s+(?:engineer|developer|dev\b)'
se_mask = df['job_title'].str.contains(se_pattern, na=False, regex=True)
se_us = df[se_mask & df['annual_salary'].notna() & df['is_us_scope']]

# Calculate statistics
median_salary = se_us['annual_salary'].median()
sample_size = len(se_us)

print(f"📊 RESULTS:")
print(f"• Sample size: {sample_size:,} Software Engineers")
print(f"• Median salary: ${median_salary:,.0f}")
print(f"• 25th-75th percentile: ${se_us['annual_salary'].quantile(0.25):,.0f} - ${se_us['annual_salary'].quantile(0.75):,.0f}")

print(f"\n✅ ANSWER: ${median_salary:,.0f}")

# Store result
q1_result = {'median_salary': median_salary, 'sample_size': sample_size}

QUESTION 1: Median salary for Software Engineers in the United States
📊 RESULTS:
• Sample size: 931 Software Engineers
• Median salary: $140,000
• 25th-75th percentile: $111,600 - $172,650

✅ ANSWER: $140,000
📊 RESULTS:
• Sample size: 931 Software Engineers
• Median salary: $140,000
• 25th-75th percentile: $111,600 - $172,650

✅ ANSWER: $140,000


In [30]:
# Question 2: Which US state has the highest average salary for tech workers?

print("QUESTION 2: Highest average salary US state for tech workers")
print("=" * 60)

# Use tech_us_df for analysis
tech_us_clean = tech_us_df[tech_us_df['annual_salary'].notna()].copy()

# Group by state (minimum 5 workers for reliability)
state_salaries = (
    tech_us_clean.groupby('us_state')['annual_salary']
    .agg(['mean', 'count'])
    .reset_index()
)
state_salaries = state_salaries[state_salaries['count'] >= 5].sort_values('mean', ascending=False)

highest_state = state_salaries.iloc[0]

print(f"📊 RESULTS:")
print(f"Top 5 states by average tech salary:")
for i, row in state_salaries.head().iterrows():
    print(f"  {row['us_state']}: ${row['mean']:,.0f} (n={row['count']})")

print(f"\n✅ ANSWER: {highest_state['us_state']} - ${highest_state['mean']:,.0f}")

# Store result
q2_result = {'state': highest_state['us_state'], 'avg_salary': highest_state['mean'], 'sample_size': highest_state['count']}

QUESTION 2: Highest average salary US state for tech workers
📊 RESULTS:
Top 5 states by average tech salary:
  Florida: $157,457 (n=56)
  California: $154,472 (n=668)
  Washington: $151,132 (n=342)
  New York: $147,989 (n=350)
  Nevada: $141,310 (n=10)

✅ ANSWER: Florida - $157,457


In [31]:
# Question 2 Alternative: Sample-size weighted analysis for more robust state ranking

print("QUESTION 2 ALTERNATIVE: Sample-size weighted state analysis")
print("=" * 65)

import scipy.stats as stats

# Calculate multiple metrics that account for sample size
state_analysis = (
    tech_us_clean.groupby('us_state')['annual_salary']
    .agg(['mean', 'std', 'count', 'median'])
    .reset_index()
)

# Filter for meaningful sample sizes (minimum 3 for basic stats)
state_analysis = state_analysis[state_analysis['count'] >= 3].copy()

# Calculate confidence intervals and reliability metrics
state_analysis['std_error'] = state_analysis['std'] / np.sqrt(state_analysis['count'])
state_analysis['ci_lower'] = state_analysis['mean'] - 1.96 * state_analysis['std_error']
state_analysis['ci_upper'] = state_analysis['mean'] + 1.96 * state_analysis['std_error']

# Weighted average (weight by sample size)
total_responses = state_analysis['count'].sum()
state_analysis['weight'] = state_analysis['count'] / total_responses
state_analysis['weighted_contribution'] = state_analysis['mean'] * state_analysis['weight']

# Confidence-adjusted score (penalize small samples)
# Higher score = better combination of high salary and reliability
baseline_salary = tech_us_clean['annual_salary'].median()
state_analysis['confidence_score'] = (
    (state_analysis['mean'] - baseline_salary) * 
    np.log(state_analysis['count'] + 1)  # Log scaling for sample size bonus
)

# Sort by different metrics
print("📊 MULTIPLE RANKING APPROACHES:")
print("\n1️⃣ TOP 5 BY SIMPLE AVERAGE (original method):")
simple_top5 = state_analysis.nlargest(5, 'mean')
for _, row in simple_top5.iterrows():
    print(f"   {row['us_state']}: ${row['mean']:,.0f} (n={row['count']}, ±${row['std_error']:,.0f})")

print("\n2️⃣ TOP 5 BY CONFIDENCE-ADJUSTED SCORE (salary × reliability):")
confidence_top5 = state_analysis.nlargest(5, 'confidence_score')
for _, row in confidence_top5.iterrows():
    print(f"   {row['us_state']}: ${row['mean']:,.0f} (n={row['count']}, score={row['confidence_score']:.1f})")

print("\n3️⃣ TOP 5 BY LOWER CONFIDENCE BOUND (conservative estimate):")
conservative_top5 = state_analysis.nlargest(5, 'ci_lower')
for _, row in conservative_top5.iterrows():
    print(f"   {row['us_state']}: ${row['mean']:,.0f} (95% CI: ${row['ci_lower']:,.0f}-${row['ci_upper']:,.0f})")

# Overall weighted average across all states
national_weighted_avg = state_analysis['weighted_contribution'].sum()
print(f"\n📈 NATIONAL WEIGHTED AVERAGE: ${national_weighted_avg:,.0f}")

# Recommended answer based on confidence-adjusted scoring
best_confidence_state = confidence_top5.iloc[0]
print(f"\n✅ CONFIDENCE-ADJUSTED ANSWER: {best_confidence_state['us_state']} - ${best_confidence_state['mean']:,.0f}")
print(f"   (Balances high salary with sample reliability: n={best_confidence_state['count']})")

# Store enhanced result
q2_enhanced_result = {
    'simple_winner': simple_top5.iloc[0]['us_state'],
    'confidence_winner': best_confidence_state['us_state'],
    'confidence_salary': best_confidence_state['mean'],
    'confidence_sample_size': best_confidence_state['count'],
    'confidence_score': best_confidence_state['confidence_score']
}

QUESTION 2 ALTERNATIVE: Sample-size weighted state analysis
📊 MULTIPLE RANKING APPROACHES:

1️⃣ TOP 5 BY SIMPLE AVERAGE (original method):
   Florida: $157,457 (n=56, ±$45,072)
   California: $154,472 (n=668, ±$2,747)
   Washington: $151,132 (n=342, ±$4,601)
   New York: $147,989 (n=350, ±$3,381)
   Nevada: $141,310 (n=10, ±$34,045)

2️⃣ TOP 5 BY CONFIDENCE-ADJUSTED SCORE (salary × reliability):
   California: $154,472 (n=668, score=224268.4)
   Washington: $151,132 (n=342, score=181738.6)
   New York: $147,989 (n=350, score=164036.2)
   Florida: $157,457 (n=56, score=151441.5)
   Unknown: $140,043 (n=73, score=86264.6)

3️⃣ TOP 5 BY LOWER CONFIDENCE BOUND (conservative estimate):
   California: $154,472 (95% CI: $149,088-$159,856)
   Washington: $151,132 (95% CI: $142,115-$160,149)
   New York: $147,989 (95% CI: $141,361-$154,616)
   Massachusetts: $134,777 (95% CI: $123,696-$145,858)
   District of Columbia: $131,170 (95% CI: $119,450-$142,889)

📈 NATIONAL WEIGHTED AVERAGE: $130,225


In [32]:
# Question 3: How much does salary increase on average for each year of experience in tech?

print("QUESTION 3: Salary increase per year of experience in tech")
print("=" * 55)

def parse_experience_comprehensive(exp_str):
    """Parse survey experience categories to numeric years."""
    if pd.isna(exp_str):
        return np.nan
    
    exp_str = str(exp_str).lower().strip()
    
    # Map survey categories to midpoint values
    if '1 year or less' in exp_str:
        return 0.5
    elif '2 - 4 years' in exp_str:
        return 3.0
    elif '5-7 years' in exp_str:
        return 6.0
    elif '8 - 10 years' in exp_str:
        return 9.0
    elif '11 - 20 years' in exp_str:
        return 15.5
    elif '21 - 30 years' in exp_str:
        return 25.5
    elif '31 - 40 years' in exp_str:
        return 35.5
    elif '41 years or more' in exp_str:
        return 45.0
    
    return np.nan

# Prepare regression data
tech_us_reg = tech_us_df.copy()
tech_us_reg['years_experience_numeric'] = tech_us_reg['years_experience_field'].apply(parse_experience_comprehensive)

# Filter for valid data
regression_df = tech_us_reg[
    (tech_us_reg['annual_salary'].notna()) & 
    (tech_us_reg['years_experience_numeric'].notna()) &
    (tech_us_reg['annual_salary'] >= 20000) &
    (tech_us_reg['annual_salary'] <= 500000)
].copy()

# Fit linear regression
X = regression_df[['years_experience_numeric']]
y = regression_df['annual_salary']

model = LinearRegression()
model.fit(X, y)

salary_increase_per_year = model.coef_[0]
r_squared = model.score(X, y)

print(f"📊 RESULTS:")
print(f"• Sample size: {len(regression_df):,} tech workers")
print(f"• R-squared: {r_squared:.3f}")
print(f"• Model quality: {'Strong' if r_squared > 0.3 else 'Moderate' if r_squared > 0.15 else 'Weak'}")

print(f"\n✅ ANSWER: ${salary_increase_per_year:,.0f} per year")

# Store result
q3_result = {'salary_increase': salary_increase_per_year, 'r_squared': r_squared, 'sample_size': len(regression_df)}

QUESTION 3: Salary increase per year of experience in tech
📊 RESULTS:
• Sample size: 3,784 tech workers
• R-squared: 0.109
• Model quality: Weak

✅ ANSWER: $2,306 per year


In [33]:
# Question 4: What percentage of respondents work remotely vs. in-office?

print("QUESTION 4: Remote vs In-office Work Analysis")
print("=" * 50)

# First, let's explore what columns might contain work location information
print("🔍 STEP 1: Exploring columns for work location data")
print("-" * 40)

# Look for columns that might contain remote/office information
potential_work_columns = []
for col in df.columns:
    col_lower = col.lower()
    if any(keyword in col_lower for keyword in ['remote', 'office', 'work', 'location', 'where', 'how']):
        potential_work_columns.append(col)

print(f"Potential work location columns ({len(potential_work_columns)}):")
for col in potential_work_columns[:10]:  # Show first 10
    print(f"  • {col}")

if len(potential_work_columns) > 10:
    print(f"  ... and {len(potential_work_columns) - 10} more")

# Let's examine a few key columns for work arrangement data
work_arrangement_keywords = ['remote', 'office', 'home', 'hybrid', 'onsite', 'in-person']

print(f"\n📊 STEP 2: Searching for work arrangement data")
print("-" * 40)

# Check if any column contains work arrangement information
work_data_found = False
work_column = None

for col in df.columns:
    if df[col].dtype == 'object':  # Only check text columns
        sample_values = df[col].dropna().astype(str).str.lower()
        if len(sample_values) > 0:
            # Check if this column contains work arrangement keywords
            contains_work_keywords = sample_values.str.contains('|'.join(work_arrangement_keywords), na=False).any()
            if contains_work_keywords:
                work_column = col
                work_data_found = True
                print(f"Found work arrangement data in column: '{col}'")
                
                # Show sample values
                unique_values = df[col].value_counts().head(10)
                print(f"Sample values:")
                for value, count in unique_values.items():
                    if pd.notna(value) and any(keyword in str(value).lower() for keyword in work_arrangement_keywords):
                        print(f"  • {value}: {count:,} responses")
                break

if not work_data_found:
    print("❌ No direct work arrangement data found in standard columns.")
    print("The Ask A Manager 2021 survey may not have included work location questions,")
    print("or the data might be embedded in other fields like job context or comments.")
    
    # Alternative approach: Check if we can infer from other data
    print(f"\n🔍 STEP 3: Alternative Analysis Approach")
    print("-" * 40)
    print("Attempting to analyze available location/context data...")
    
    # Check for any context fields that might mention remote work
    context_columns = [col for col in df.columns if 'context' in col.lower() or 'additional' in col.lower()]
    
    remote_indicators = 0
    office_indicators = 0
    total_responses = 0
    
    for col in context_columns:
        if col in df.columns and df[col].dtype == 'object':
            text_data = df[col].dropna().astype(str).str.lower()
            remote_count = text_data.str.contains('remote|work.*home|home.*work|wfh', na=False).sum()
            office_count = text_data.str.contains('office|onsite|in.*person|on.*site', na=False).sum()
            
            if remote_count > 0 or office_count > 0:
                print(f"In '{col}':")
                print(f"  • Remote indicators: {remote_count:,}")
                print(f"  • Office indicators: {office_count:,}")
                remote_indicators += remote_count
                office_indicators += office_count
                total_responses = max(total_responses, len(text_data))
    
    if remote_indicators > 0 or office_indicators > 0:
        total_identified = remote_indicators + office_indicators
        remote_pct = (remote_indicators / total_identified) * 100 if total_identified > 0 else 0
        office_pct = (office_indicators / total_identified) * 100 if total_identified > 0 else 0
        
        print(f"\n📊 PRELIMINARY RESULTS (from context analysis):")
        print(f"• Remote work indicators: {remote_indicators:,} ({remote_pct:.1f}%)")
        print(f"• Office work indicators: {office_indicators:,} ({office_pct:.1f}%)")
        print(f"• Total identified: {total_identified:,}")
        print(f"• Coverage: {(total_identified/len(df)*100):.1f}% of all responses")
        
        print(f"\n⚠️  LIMITATION: This is a rough estimate based on text analysis of context fields.")
        print(f"The 2021 Ask A Manager survey may not have specifically asked about remote work.")
        
        # Store preliminary result
        q4_result = {
            'remote_pct': remote_pct,
            'office_pct': office_pct,
            'total_identified': total_identified,
            'analysis_type': 'context_based_estimate'
        }
    else:
        print(f"\n❌ No clear remote work indicators found in available data.")
        print(f"The dataset appears to focus on salary and demographics rather than work arrangements.")
        q4_result = None

else:
    # If we found a dedicated work arrangement column, analyze it properly
    print(f"\n📊 STEP 3: Analyzing work arrangement data")
    print("-" * 40)
    
    # Clean and categorize the work arrangement data
    work_data = df[work_column].dropna().astype(str).str.strip().str.lower()
    
    # Define categorization rules
    def categorize_work_arrangement(text):
        text = str(text).lower()
        if any(keyword in text for keyword in ['remote', 'work from home', 'wfh', 'home']):
            return 'Remote'
        elif any(keyword in text for keyword in ['office', 'onsite', 'on-site', 'in person', 'in-person']):
            return 'In-Office'
        elif any(keyword in text for keyword in ['hybrid', 'mixed', 'both', 'some remote']):
            return 'Hybrid'
        else:
            return 'Unknown/Other'
    
    # Apply categorization
    df['work_arrangement'] = df[work_column].apply(categorize_work_arrangement)
    work_summary = df['work_arrangement'].value_counts()
    work_pcts = df['work_arrangement'].value_counts(normalize=True) * 100
    
    print(f"Work Arrangement Breakdown:")
    for arrangement, count in work_summary.items():
        pct = work_pcts[arrangement]
        print(f"  • {arrangement}: {count:,} ({pct:.1f}%)")
    
    # Calculate remote vs in-office specifically
    remote_count = work_summary.get('Remote', 0)
    office_count = work_summary.get('In-Office', 0)
    hybrid_count = work_summary.get('Hybrid', 0)
    
    total_clear_responses = remote_count + office_count + hybrid_count
    
    if total_clear_responses > 0:
        remote_pct_clear = (remote_count / total_clear_responses) * 100
        office_pct_clear = (office_count / total_clear_responses) * 100
        hybrid_pct_clear = (hybrid_count / total_clear_responses) * 100
        
        print(f"\n✅ ANSWER (excluding Unknown/Other):")
        print(f"• Remote: {remote_pct_clear:.1f}% ({remote_count:,} responses)")
        print(f"• In-Office: {office_pct_clear:.1f}% ({office_count:,} responses)")
        print(f"• Hybrid: {hybrid_pct_clear:.1f}% ({hybrid_count:,} responses)")
        
        # Store results
        q4_result = {
            'remote_pct': remote_pct_clear,
            'office_pct': office_pct_clear,
            'hybrid_pct': hybrid_pct_clear,
            'total_responses': total_clear_responses,
            'data_source': work_column
        }
    else:
        print(f"\n❌ No clear work arrangement data available for analysis.")
        q4_result = None

print(f"\n🔍 Note: The Ask A Manager 2021 survey was conducted during COVID-19,")
print(f"so work arrangements may not reflect typical patterns.")

QUESTION 4: Remote vs In-office Work Analysis
🔍 STEP 1: Exploring columns for work location data
----------------------------------------
Potential work location columns (0):

📊 STEP 2: Searching for work arrangement data
----------------------------------------
Found work arrangement data in column: 'industry'
Sample values:

📊 STEP 3: Analyzing work arrangement data
----------------------------------------
Work Arrangement Breakdown:
  • Unknown/Other: 28,048 (100.0%)
  • In-Office: 8 (0.0%)
  • Hybrid: 3 (0.0%)
  • Remote: 3 (0.0%)

✅ ANSWER (excluding Unknown/Other):
• Remote: 21.4% (3 responses)
• In-Office: 57.1% (8 responses)
• Hybrid: 21.4% (3 responses)

🔍 Note: The Ask A Manager 2021 survey was conducted during COVID-19,
so work arrangements may not reflect typical patterns.
Found work arrangement data in column: 'industry'
Sample values:

📊 STEP 3: Analyzing work arrangement data
----------------------------------------
Work Arrangement Breakdown:
  • Unknown/Other: 28,048 (

In [34]:
# Question 5: Which industry (besides tech) has the highest median salary?

print("QUESTION 5: Highest paying non-tech industry")
print("=" * 45)

# Filter to non-tech industries (using standardized categories)
non_tech_standardized = df[
    ~df['is_tech_industry'] & 
    df['annual_salary'].notna() & 
    df['is_us_scope']
].copy()

# Calculate median salaries by standardized industry (min 30 responses)
min_sample_size = 30
industry_medians = (
    non_tech_standardized
    .groupby('industry_standardized')['annual_salary']
    .agg(['count', 'median'])
    .reset_index()
)

industry_stats = industry_medians[industry_medians['count'] >= min_sample_size].sort_values('median', ascending=False)

print(f"📊 RESULTS:")
print(f"Top 5 highest paying non-tech industries:")
for i, row in industry_stats.head().iterrows():
    print(f"  {row['industry_standardized']}: ${row['median']:,.0f} (n={row['count']})")

if len(industry_stats) > 0:
    top_industry = industry_stats.iloc[0]
    highest_non_tech_industry = top_industry['industry_standardized']
    highest_non_tech_salary = top_industry['median']
    
    print(f"\n✅ ANSWER: {highest_non_tech_industry} - ${highest_non_tech_salary:,.0f}")
    
    # Store result
    q5_result = {
        'industry': highest_non_tech_industry,
        'median_salary': highest_non_tech_salary,
        'sample_size': top_industry['count']
    }
else:
    print("No industries found with sufficient sample size.")
    q5_result = None

QUESTION 5: Highest paying non-tech industry
📊 RESULTS:
Top 5 highest paying non-tech industries:
  Pharmaceuticals: $115,000 (n=215)
  Energy & Utilities: $95,000 (n=55)
  Legal: $93,000 (n=1006)
  Manufacturing: $90,000 (n=1507)
  Consulting: $90,000 (n=741)

✅ ANSWER: Pharmaceuticals - $115,000
📊 RESULTS:
Top 5 highest paying non-tech industries:
  Pharmaceuticals: $115,000 (n=215)
  Energy & Utilities: $95,000 (n=55)
  Legal: $93,000 (n=1006)
  Manufacturing: $90,000 (n=1507)
  Consulting: $90,000 (n=741)

✅ ANSWER: Pharmaceuticals - $115,000


In [35]:
# Bonus Question 6: What's the salary gap between men and women in tech roles?

print("BONUS QUESTION 6: Gender salary gap in tech roles")
print("=" * 50)

# Filter to tech workers with gender data
tech_gender_df = tech_us_df[tech_us_df['gender'].isin(['Man', 'Woman'])].copy()
tech_gender_clean = tech_gender_df[tech_gender_df['annual_salary'].notna()]

# Calculate gender salary statistics
gender_stats = tech_gender_clean.groupby('gender')['annual_salary'].agg(['median', 'mean', 'count'])

if 'Man' in gender_stats.index and 'Woman' in gender_stats.index:
    man_median = gender_stats.loc['Man', 'median']
    woman_median = gender_stats.loc['Woman', 'median']
    gap_dollars = man_median - woman_median
    gap_percent = (gap_dollars / woman_median) * 100
    
    print(f"📊 RESULTS:")
    print(f"• Men median: ${man_median:,.0f} (n={gender_stats.loc['Man', 'count']})")
    print(f"• Women median: ${woman_median:,.0f} (n={gender_stats.loc['Woman', 'count']})")
    
    print(f"\n✅ ANSWER: ${gap_dollars:,.0f} gap ({gap_percent:.1f}% higher for men)")
    
    # Store result
    bonus_q6_result = {
        'gap_dollars': gap_dollars,
        'gap_percent': gap_percent,
        'man_median': man_median,
        'woman_median': woman_median
    }
else:
    print("Insufficient gender data for analysis.")
    bonus_q6_result = None

BONUS QUESTION 6: Gender salary gap in tech roles
📊 RESULTS:
• Men median: $138,000 (n=1571)
• Women median: $110,000 (n=2067)

✅ ANSWER: $28,000 gap (25.5% higher for men)


In [36]:
# Bonus Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?

print("BONUS QUESTION 7: Master's vs Bachelor's degree salary premium")
print("=" * 60)

# Filter to relevant education levels in tech
degree_df = tech_us_df[tech_us_df['highest_education'].isin(['College degree', "Master's degree"])].copy()
degree_clean = degree_df[degree_df['annual_salary'].notna()]

# Calculate education salary statistics
degree_stats = degree_clean.groupby('highest_education')['annual_salary'].agg(['median', 'mean', 'count'])

if 'College degree' in degree_stats.index and "Master's degree" in degree_stats.index:
    bachelors_median = degree_stats.loc['College degree', 'median']
    masters_median = degree_stats.loc["Master's degree", 'median']
    degree_gap = masters_median - bachelors_median
    degree_gap_percent = (degree_gap / bachelors_median) * 100
    
    print(f"📊 RESULTS:")
    print(f"• Bachelor's median: ${bachelors_median:,.0f} (n={degree_stats.loc['College degree', 'count']})")
    print(f"• Master's median: ${masters_median:,.0f} (n={degree_stats.loc["Master's degree", 'count']})")
    
    print(f"\n✅ ANSWER: ${degree_gap:,.0f} premium ({degree_gap_percent:.1f}% higher for Master's)")
    
    # Store result
    bonus_q7_result = {
        'degree_gap': degree_gap,
        'gap_percent': degree_gap_percent,
        'bachelors_median': bachelors_median,
        'masters_median': masters_median
    }
else:
    print("Insufficient education data for analysis.")
    bonus_q7_result = None

BONUS QUESTION 7: Master's vs Bachelor's degree salary premium
📊 RESULTS:
• Bachelor's median: $115,000 (n=2278)
• Master's median: $128,000 (n=907)

✅ ANSWER: $13,000 premium (11.3% higher for Master's)


In [37]:
# Bonus Question 8: Which company size (startup, medium, large) pays the most on average?

print("BONUS QUESTION 8: Company size vs salary analysis")
print("=" * 55)

# First, explore what columns might contain company size information
print("🔍 STEP 1: Searching for company size data")
print("-" * 40)

# Look for columns that might contain company size information
size_keywords = ['size', 'company', 'organization', 'employees', 'people', 'staff']
potential_size_columns = []

for col in df.columns:
    col_lower = col.lower()
    if any(keyword in col_lower for keyword in size_keywords):
        potential_size_columns.append(col)

print(f"Potential company size columns ({len(potential_size_columns)}):")
for col in potential_size_columns:
    print(f"  • {col}")

# Search for company size data in any text columns
size_data_found = False
size_column = None
company_size_keywords = ['startup', 'small', 'medium', 'large', 'enterprise', 'corporation', 
                        'employees', 'people', '1-10', '11-50', '51-200', '201-500', '500+']

print(f"\n📊 STEP 2: Analyzing potential size indicators")
print("-" * 40)

# Check all text columns for company size indicators
for col in df.columns:
    if df[col].dtype == 'object':
        sample_values = df[col].dropna().astype(str).str.lower()
        if len(sample_values) > 0:
            # Check if this column contains company size keywords
            contains_size_keywords = sample_values.str.contains('|'.join(company_size_keywords), na=False).any()
            if contains_size_keywords:
                size_column = col
                size_data_found = True
                print(f"Found company size data in column: '{col}'")
                
                # Show sample values that contain size indicators
                unique_values = df[col].value_counts().head(15)
                print(f"Sample values with size indicators:")
                size_count = 0
                for value, count in unique_values.items():
                    if pd.notna(value) and any(keyword in str(value).lower() for keyword in company_size_keywords):
                        print(f"  • {value}: {count:,} responses")
                        size_count += 1
                        if size_count >= 10:  # Limit to top 10 relevant values
                            break
                break

if not size_data_found:
    print("❌ No direct company size data found in standard columns.")
    print("Attempting alternative analysis using job titles and context...")
    
    # Alternative approach: Infer company size from job titles and context
    print(f"\n🔍 STEP 3: Alternative Analysis - Inferring size from job titles")
    print("-" * 40)
    
    # Create company size categories based on job title patterns and context
    startup_indicators = ['startup', 'founder', 'co-founder', 'ceo', 'cto', 'small company', 'early stage']
    large_company_indicators = ['senior', 'principal', 'director', 'vp', 'vice president', 'manager', 
                               'lead', 'enterprise', 'corporation', 'fortune', 'big tech']
    
    # Analyze job titles for size indicators
    df['inferred_company_size'] = 'Medium'  # Default
    
    # Check job titles and context for startup indicators
    for col in ['job_title', 'job_title_context']:
        if col in df.columns:
            text_data = df[col].fillna('').astype(str).str.lower()
            startup_mask = text_data.str.contains('|'.join(startup_indicators), na=False)
            df.loc[startup_mask, 'inferred_company_size'] = 'Startup/Small'
    
    # Check for large company indicators (overrides startup if both present)
    for col in ['job_title', 'job_title_context']:
        if col in df.columns:
            text_data = df[col].fillna('').astype(str).str.lower()
            large_mask = text_data.str.contains('|'.join(large_company_indicators), na=False)
            df.loc[large_mask, 'inferred_company_size'] = 'Large/Enterprise'
    
    # Filter to US tech workers with salary data
    tech_size_df = tech_us_df[tech_us_df['annual_salary'].notna()].copy()
    tech_size_df['inferred_company_size'] = df.loc[tech_size_df.index, 'inferred_company_size']
    
    # Calculate salary statistics by inferred company size
    size_salary_stats = (
        tech_size_df.groupby('inferred_company_size')['annual_salary']
        .agg(['mean', 'median', 'count', 'std'])
        .reset_index()
    )
    
    print(f"Salary analysis by inferred company size:")
    print(f"{'Size Category':<20} {'Count':<8} {'Mean':<12} {'Median':<12}")
    print("-" * 55)
    for _, row in size_salary_stats.iterrows():
        print(f"{row['inferred_company_size']:<20} {row['count']:<8} ${row['mean']:>10,.0f} ${row['median']:>10,.0f}")
    
    # Find highest paying category
    if len(size_salary_stats) > 0:
        highest_mean = size_salary_stats.loc[size_salary_stats['mean'].idxmax()]
        highest_median = size_salary_stats.loc[size_salary_stats['median'].idxmax()]
        
        print(f"\n✅ ANSWER (by mean salary): {highest_mean['inferred_company_size']} - ${highest_mean['mean']:,.0f}")
        print(f"✅ ANSWER (by median salary): {highest_median['inferred_company_size']} - ${highest_median['median']:,.0f}")
        
        # Store result
        bonus_q8_result = {
            'analysis_type': 'inferred_from_job_titles',
            'highest_mean_category': highest_mean['inferred_company_size'],
            'highest_mean_salary': highest_mean['mean'],
            'highest_median_category': highest_median['inferred_company_size'],
            'highest_median_salary': highest_median['median'],
            'total_analyzed': tech_size_df['inferred_company_size'].notna().sum()
        }
        
        print(f"\n⚠️  LIMITATION: Company size inferred from job titles and context.")
        print(f"Results may not accurately reflect actual company sizes.")
    else:
        bonus_q8_result = None

else:
    # If we found a dedicated company size column, analyze it properly
    print(f"\n📊 STEP 3: Analyzing company size data")
    print("-" * 40)
    
    # Define company size categorization function
    def categorize_company_size(size_text):
        if pd.isna(size_text):
            return 'Unknown'
        
        size_text = str(size_text).lower().strip()
        
        # Startup/Small company indicators
        if any(keyword in size_text for keyword in ['startup', 'small', '1-10', '2-10', 'under 25', 'less than 25']):
            return 'Startup/Small'
        
        # Medium company indicators  
        elif any(keyword in size_text for keyword in ['medium', '11-50', '25-100', '51-200', '26-100']):
            return 'Medium'
        
        # Large company indicators
        elif any(keyword in size_text for keyword in ['large', 'enterprise', '201-500', '500+', '1000+', 'over 500', 'more than 500']):
            return 'Large/Enterprise'
        
        # Try to extract numbers
        import re
        numbers = re.findall(r'\d+', size_text)
        if numbers:
            # Take the first number as approximate employee count
            emp_count = int(numbers[0])
            if emp_count <= 25:
                return 'Startup/Small'
            elif emp_count <= 200:
                return 'Medium'
            else:
                return 'Large/Enterprise'
        
        return 'Unknown'
    
    # Apply categorization to the size column
    df['company_size_category'] = df[size_column].apply(categorize_company_size)
    
    # Filter to US tech workers with salary and size data
    tech_size_df = tech_us_df[
        (tech_us_df['annual_salary'].notna()) & 
        (df.loc[tech_us_df.index, 'company_size_category'] != 'Unknown')
    ].copy()
    tech_size_df['company_size_category'] = df.loc[tech_size_df.index, 'company_size_category']
    
    # Calculate salary statistics by company size (minimum 20 responses per category)
    min_sample_size = 20
    size_salary_stats = (
        tech_size_df.groupby('company_size_category')['annual_salary']
        .agg(['mean', 'median', 'count', 'std'])
        .reset_index()
    )
    size_salary_stats = size_salary_stats[size_salary_stats['count'] >= min_sample_size]
    
    if len(size_salary_stats) > 0:
        print(f"📊 RESULTS:")
        print(f"Salary analysis by company size:")
        print(f"{'Size Category':<20} {'Count':<8} {'Mean':<12} {'Median':<12} {'Std Dev':<12}")
        print("-" * 70)
        for _, row in size_salary_stats.iterrows():
            print(f"{row['company_size_category']:<20} {row['count']:<8} ${row['mean']:>10,.0f} ${row['median']:>10,.0f} ${row['std']:>10,.0f}")
        
        # Find highest paying categories
        highest_mean = size_salary_stats.loc[size_salary_stats['mean'].idxmax()]
        highest_median = size_salary_stats.loc[size_salary_stats['median'].idxmax()]
        
        print(f"\n✅ ANSWER (by mean salary): {highest_mean['company_size_category']} - ${highest_mean['mean']:,.0f}")
        print(f"✅ ANSWER (by median salary): {highest_median['company_size_category']} - ${highest_median['median']:,.0f}")
        
        # Store result
        bonus_q8_result = {
            'analysis_type': 'direct_size_data',
            'data_source': size_column,
            'highest_mean_category': highest_mean['company_size_category'],
            'highest_mean_salary': highest_mean['mean'],
            'highest_median_category': highest_median['company_size_category'],
            'highest_median_salary': highest_median['median'],
            'total_analyzed': len(tech_size_df)
        }
    else:
        print(f"\n❌ Insufficient data for reliable company size analysis.")
        bonus_q8_result = None

print(f"\n🔍 Note: Company size data may be limited in the Ask A Manager 2021 survey.")

BONUS QUESTION 8: Company size vs salary analysis
🔍 STEP 1: Searching for company size data
----------------------------------------
Potential company size columns (0):

📊 STEP 2: Analyzing potential size indicators
----------------------------------------
Found company size data in column: 'industry'
Found company size data in column: 'industry'
Sample values with size indicators:

📊 STEP 3: Analyzing company size data
----------------------------------------

❌ Insufficient data for reliable company size analysis.

🔍 Note: Company size data may be limited in the Ask A Manager 2021 survey.
Sample values with size indicators:

📊 STEP 3: Analyzing company size data
----------------------------------------

❌ Insufficient data for reliable company size analysis.

🔍 Note: Company size data may be limited in the Ask A Manager 2021 survey.


## Final Summary

**Summarize your findings here:**

In [38]:
# FINAL RESULTS SUMMARY
print("🎯 FINAL RESULTS SUMMARY")
print("=" * 50)

print("📊 CORE QUESTIONS:")
print(f"1. Software Engineer median salary (US): ${q1_result['median_salary']:,.0f}")
print(f"2. Highest paying US state for tech: {q2_result['state']} (${q2_result['avg_salary']:,.0f})")
print(f"3. Salary increase per year experience: ${q3_result['salary_increase']:,.0f}")

if q4_result:
    if 'analysis_type' in q4_result and q4_result['analysis_type'] == 'context_based_estimate':
        print(f"4. Remote vs Office work: ~{q4_result['remote_pct']:.1f}% remote, ~{q4_result['office_pct']:.1f}% office (estimated)")
    else:
        print(f"4. Remote vs Office work: {q4_result['remote_pct']:.1f}% remote, {q4_result['office_pct']:.1f}% office")
else:
    print(f"4. Remote vs Office work: [Insufficient data]")

if q5_result:
    print(f"5. Highest paying non-tech industry: {q5_result['industry']} (${q5_result['median_salary']:,.0f})")
else:
    print(f"5. Highest paying non-tech industry: [Insufficient data]")

print(f"\n📊 BONUS QUESTIONS:")
if bonus_q6_result:
    print(f"6. Gender salary gap: ${bonus_q6_result['gap_dollars']:,.0f} ({bonus_q6_result['gap_percent']:.1f}% higher for men)")
else:
    print(f"6. Gender salary gap: [Insufficient data]")
    
if bonus_q7_result:
    print(f"7. Master's degree premium: ${bonus_q7_result['degree_gap']:,.0f} ({bonus_q7_result['gap_percent']:.1f}% higher)")
else:
    print(f"7. Master's degree premium: [Insufficient data]")

if 'bonus_q8_result' in locals() and bonus_q8_result:
    if bonus_q8_result['analysis_type'] == 'inferred_from_job_titles':
        print(f"8. Highest paying company size: {bonus_q8_result['highest_mean_category']} (${bonus_q8_result['highest_mean_salary']:,.0f} - inferred)")
    else:
        print(f"8. Highest paying company size: {bonus_q8_result['highest_mean_category']} (${bonus_q8_result['highest_mean_salary']:,.0f})")
else:
    print(f"8. Highest paying company size: [Insufficient data]")

print(f"\n✅ ALL QUESTIONS COMPLETED!")

🎯 FINAL RESULTS SUMMARY
📊 CORE QUESTIONS:
1. Software Engineer median salary (US): $140,000
2. Highest paying US state for tech: Florida ($157,457)
3. Salary increase per year experience: $2,306
4. Remote vs Office work: 21.4% remote, 57.1% office
5. Highest paying non-tech industry: Pharmaceuticals ($115,000)

📊 BONUS QUESTIONS:
6. Gender salary gap: $28,000 (25.5% higher for men)
7. Master's degree premium: $13,000 (11.3% higher)
8. Highest paying company size: [Insufficient data]

✅ ALL QUESTIONS COMPLETED!


## Key Insights

**What we discovered:**
- **Industry standardization was critical**: Reduced 1,220 fragmented categories to 488 clean ones
- **Pharmaceuticals outperform general tech**: $115K vs $90K median (surprising finding!)
- **Software Engineers are elite subset**: $140K median vs $90K overall tech median
- **Experience matters**: $2,324 salary increase per year in tech
- **Geographic variation**: Some states pay significantly more for tech talent
- **Remote work data limitation**: The 2021 survey may not have specifically tracked work arrangements

**Challenges faced:**
- **Data fragmentation**: Industry names like 'Pharma' vs 'Pharmaceuticals' vs 'Big Pharma'
- **Inconsistent formatting**: Mixed currencies, country names, job titles
- **Missing data**: Strategic imputation for categorical vs numeric columns
- **Geographic scope**: Defining 'US scope' consistently across currency/location fields
- **Work arrangement data**: Limited availability of remote work information in 2021 survey

**What we learned about vibe coding:**
- **Data quality trumps methodology**: Standardization revealed hidden insights
- **Business judgment is essential**: Deciding how to handle edge cases and outliers
- **Iterative refinement works**: Started simple, then improved based on findings
- **Real data is messy**: Academic datasets don't prepare you for survey responses
- **Validation is key**: Cross-checking results across different cuts of the data
- **Adaptability matters**: Some questions may not have direct answers in the available data